In [10]:
from osgeo import gdal #to crop tile
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv

# File paths to images
from pathlib import Path

In [2]:
# Split every image into 54x54 px to create around 256x256m images

planetscope_mosaic = '/Users/rjlee6/Documents/ethiopia/tigray_north/'
# output_path = '/Users/rjlee6/Documents/ethiopia/tigray_north_256'

data_to_process = glob.glob(os.path.join(planetscope_mosaic, '*.tif'))

print(len(data_to_process))

840


In [16]:
# Split every image into 55x55 px to create around 256x256m images
planetscope_mosaic = '/Users/rjlee6/Documents/ethiopia/tigray_north/'
output_path = '/Users/rjlee6/Documents/ethiopia/tigray_north_256'

data_to_process = glob.glob(os.path.join(planetscope_mosaic, '*.tif'))
tile_px = 54
# Start 0-200 (done)
# 200-400 (done)
# 400-600
# 600-840

# Define the EPSG codes for the coordinate reference systems
source_crs = pyproj.CRS('EPSG:3857')
target_crs = pyproj.CRS('EPSG:4326')

# Define a transformer object to convert coordinates from source to target CRS
transformer = pyproj.Transformer.from_crs(source_crs, target_crs)

with open('/Users/rjlee6/Documents/ethiopia/tigray_north_256_center.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['filename', 'x_wgs', 'y_wgs'])

    for mosaic in data_to_process:

        img = gdal.Open(mosaic)
        gt = img.GetGeoTransform()
        # Upper left coordinates
        xmin = gt[0]
        ymax = gt[3]

        # Resolution and division
        res = gt[1]
        tile_size = res * tile_px
        num_x = int(img.RasterXSize / tile_px)
        num_y = int(img.RasterYSize / tile_px)

        xsteps = [xmin + tile_size * i for i in range(num_x+1)]
        ysteps = [ymax - tile_size * i for i in range(num_y+1)]

        for i in range(num_x):
            for j in range(num_y):
                xmin = xsteps[i]
                xmax = xsteps[i+1]
                ymax = ysteps[j]
                ymin = ysteps[j+1]
                center_x = (xmin + xmax) / 2
                center_y = (ymin + ymax) / 2

                # Transform the center coordinates to WGS84
                x_wgs, y_wgs = transformer.transform(center_x, center_y)
                # Create and save the subsetted image
                # filename = output_path + mosaic[45:-4] + '_' + str(i).zfill(2) + str(j).zfill(2) + '.tif'
                filename = mosaic[46:-4] + '_' + str(i).zfill(2) + str(j).zfill(2) + '.tif'

                writer.writerow([filename, x_wgs, y_wgs])
        
                # raster = gdal.Warp(filename, img, outputBounds=(xmin,ymin,xmax,ymax))
                # print(filename)

In [8]:
# Check the length of tigray_north_256
path_256 = '/Users/rjlee6/Documents/ethiopia/tigray_north_256/'
data_to_process = glob.glob(os.path.join(path_256, '*.tif'))
print(len(data_to_process))

840


In [ ]:
# Delete all images that don't contain full image (from skysat collect)
clip_path = '/Users/rjlee6/Documents/ethiopia/tigray_north_256'
data_to_process = glob.glob(os.path.join(clip_path, '*.tif'))
for clip_img in data_to_process[0:100]:
    img = gdal.Open(clip_img)
    band = img.GetRasterBand(4)
    img_array = band.ReadAsArray()
    if 0 in img_array:
        os.remove(clip_img)
        print('Deleted ' + clip_img)

In [ ]:
# Test and see the dimensions of a skysat_loc planetscope image
test_planetscope_img = '/Users/rjlee6/Documents/ethiopia/tigray_north_clip/1_2021_01_L15-1234E-1104N_0000.tif'
test = gdal.Open(test_planetscope_img)

print(test.GetGeoTransform())

def load_geotiff(file):
    ds = gdal.Open(file)
    band1 = ds.GetRasterBand(1).ReadAsArray() # Red channel
    band2 = ds.GetRasterBand(2).ReadAsArray() # Green channel
    band3 = ds.GetRasterBand(3).ReadAsArray() # Blue channel
    img = np.dstack((band1, band2, band3))
    return img
image = load_geotiff(test_planetscope_img)
print(image.shape)
plt.imshow(image)

In [ ]:
# Test and see the dimensions of a skysat_loc planetscope image
test_planetscope_img = '/Users/rjlee6/Downloads/20201101_074640_ssc2_u0001_visual_clip_01_0101.tiff'
test = gdal.Open(test_planetscope_img)

print(test.GetGeoTransform())

def load_geotiff(file):
    ds = gdal.Open(file)
    band1 = ds.GetRasterBand(1).ReadAsArray() # Red channel
    band2 = ds.GetRasterBand(2).ReadAsArray() # Green channel
    band3 = ds.GetRasterBand(3).ReadAsArray() # Blue channel
    img = np.dstack((band1, band2, band3))
    return img
image = load_geotiff(test_planetscope_img)
print(image.shape)
plt.imshow(image)